In [1]:
!pwd

/Users/ashish1610dhiman/data_projects/bestbuy/notebooks/ashish


In [2]:
import sys
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

sys.version

sys.path.append("../../")

from src.ad_hmm import sku_predict
from src.utils import *

In [3]:
%load_ext autoreload 
%autoreload 2

### Input Data

In [4]:
train_test = pd.read_csv("../../data/train_validation_marker.csv")
train_test["SALES_DATE"] = pd.to_datetime(train_test["SALES_DATE"])
print (train_test.shape)

train_test["if_promo"] = (~train_test["PROMO_PRICE"].isin(["?"])).astype("int")
train_test["if_competitor"] = (~train_test["COMPETITOR_PRICE"].isin(["?"])).astype("int")

train_test["min_sales_price"] = pd.to_numeric(np.where(train_test.if_promo>0, train_test.PROMO_PRICE,\
                                                       train_test.RETAIL_PRICE))

train_test["comp_price_clean"] = pd.to_numeric(np.where(train_test.if_competitor>0,train_test.COMPETITOR_PRICE, np.nan))
train_test["price_delta"] = np.where(train_test.if_competitor>0, train_test.min_sales_price - train_test.comp_price_clean,\
                                   -train_test.min_sales_price)

(846632, 15)


In [5]:
train_test_pivot = train_test.pivot(index='SALES_DATE', columns='Encoded_SKU_ID',\
                   values=['DAILY_UNITS',"if_promo","if_competitor","min_sales_price","price_delta",\
                          "validation","validation_clean","RETAIL_PRICE"])
train_test_pivot.columns = train_test_pivot.columns.swaplevel(0, 1)

### Test for one SKU

In [6]:
sku_id = 50
train_test_sku = train_test_pivot.loc[:,(sku_id,)].copy()
train_test_sku["ad_time"] = list(range(1,train_test_sku.shape[0]+1))

/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36436/4124826544.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
  train_test_sku = train_test_pivot.loc[:,(sku_id,)].copy()


In [7]:
train_sku = train_test_sku[(train_test_sku.validation==False) & (train_test_sku.validation_clean==True)]
validation_sku = train_test_sku[(train_test_sku.validation==True) & (train_test_sku.validation_clean==True)]
train_sku.shape,validation_sku.shape

((1520, 9), (7, 9))

In [8]:
from statsmodels.tsa.seasonal import STL,MSTL
res = STL(train_sku["DAILY_UNITS"], period = 365).fit()
res_m = MSTL(train_sku["DAILY_UNITS"], periods=(7, 30, 365)).fit()

In [9]:
train_sku["STL_resid"] = res.resid
train_sku["STL_trend"] = res.trend
train_sku["STL_seasonal"] = res.seasonal

train_sku["MSTL_resid"] = res_m.resid
train_sku["MSTL_trend"] = res_m.trend
train_sku[["MSTL_seasonal_7","MSTL_seasonal_30","MSTL_seasonal_365"]] = res_m.seasonal

/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36436/1610264522.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_sku["STL_resid"] = res.resid
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36436/1610264522.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_sku["STL_trend"] = res.trend
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36436/1610264522.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

### try Prophet

In [10]:
from prophet import Prophet

Importing plotly failed. Interactive plots will not work.


In [11]:
pred_cols = ['if_promo', 'if_competitor', 'min_sales_price',
       'price_delta',"ad_time","RETAIL_PRICE"]

In [12]:
model = Prophet()
model.add_country_holidays(country_name='US')
for col in pred_cols:
    model.add_regressor(col)

In [13]:
X_train1 = train_sku[pred_cols+["DAILY_UNITS"]].astype(np.float64)
X_test1 = validation_sku[pred_cols].astype(np.float64)

In [14]:
X_train_prophet = X_train1.reset_index().rename(columns={"SALES_DATE":"ds",\
                                                       "DAILY_UNITS":"y"})
X_test_prophet = X_test1.reset_index().rename(columns={"SALES_DATE":"ds"})
X_train_prophet

,ds,if_promo,if_competitor,min_sales_price,price_delta,ad_time,RETAIL_PRICE,y
0,2018-05-27,0.0,0.0,99.99,-99.99,365.0,99.99,1.0
1,2018-05-28,0.0,0.0,99.99,-99.99,366.0,99.99,1.0
2,2018-05-29,0.0,0.0,99.99,-99.99,367.0,99.99,0.0
3,2018-05-30,0.0,0.0,99.99,-99.99,368.0,99.99,0.0
4,2018-05-31,0.0,0.0,99.99,-99.99,369.0,99.99,10.0
...,...,...,...,...,...,...,...,...
1515,2022-07-20,1.0,0.0,79.99,-79.99,1880.0,104.99,70.0
1516,2022-07-21,1.0,0.0,79.99,-79.99,1881.0,104.99,62.0
1517,2022-07-22,1.0,0.0,79.99,-79.99,1882.0,104.99,84.0
1518,2022-07-23,1.0,0.0,79.99,-79.99,1883.0,104.99,75.0


In [15]:
model.fit(X_train_prophet)

21:54:37 - cmdstanpy - INFO - Chain [1] start processing
21:54:37 - cmdstanpy - INFO - Chain [1] done processing


In [16]:
test_pred = model.predict(X_test_prophet)

In [17]:
test_pred.columns

Index(['ds', 'trend', 'yhat_lower', 'yhat_upper', 'trend_lower', 'trend_upper',
       'Christmas Day', 'Christmas Day_lower', 'Christmas Day_upper',
       'Christmas Day (Observed)', 'Christmas Day (Observed)_lower',
       'Christmas Day (Observed)_upper', 'Columbus Day', 'Columbus Day_lower',
       'Columbus Day_upper', 'Independence Day', 'Independence Day_lower',
       'Independence Day_upper', 'Independence Day (Observed)',
       'Independence Day (Observed)_lower',
       'Independence Day (Observed)_upper',
       'Juneteenth National Independence Day',
       'Juneteenth National Independence Day_lower',
       'Juneteenth National Independence Day_upper',
       'Juneteenth National Independence Day (Observed)',
       'Juneteenth National Independence Day (Observed)_lower',
       'Juneteenth National Independence Day (Observed)_upper', 'Labor Day',
       'Labor Day_lower', 'Labor Day_upper', 'Martin Luther King Jr. Day',
       'Martin Luther King Jr. Day_lower', 'Mart

In [18]:
test_pred["yhat"]

0    28.278024
1    29.129796
2    30.470059
3    32.552071
4    40.340107
5    46.906114
6    30.437583
Name: yhat, dtype: float64

In [19]:
validation_sku["DAILY_UNITS"]

SALES_DATE
2022-07-25    25
2022-07-26    33
2022-07-27    44
2022-07-28    49
2022-07-29    43
2022-07-30    55
2022-07-31    45
Name: DAILY_UNITS, dtype: object

In [20]:
train_pred = model.predict(X_train_prophet)

In [28]:
train_pred.columns

Index(['ds', 'trend', 'yhat_lower', 'yhat_upper', 'trend_lower', 'trend_upper',
       'Christmas Day', 'Christmas Day_lower', 'Christmas Day_upper',
       'Christmas Day (Observed)', 'Christmas Day (Observed)_lower',
       'Christmas Day (Observed)_upper', 'Columbus Day', 'Columbus Day_lower',
       'Columbus Day_upper', 'Independence Day', 'Independence Day_lower',
       'Independence Day_upper', 'Independence Day (Observed)',
       'Independence Day (Observed)_lower',
       'Independence Day (Observed)_upper',
       'Juneteenth National Independence Day',
       'Juneteenth National Independence Day_lower',
       'Juneteenth National Independence Day_upper',
       'Juneteenth National Independence Day (Observed)',
       'Juneteenth National Independence Day (Observed)_lower',
       'Juneteenth National Independence Day (Observed)_upper', 'Labor Day',
       'Labor Day_lower', 'Labor Day_upper', 'Martin Luther King Jr. Day',
       'Martin Luther King Jr. Day_lower', 'Mart

### Feature eng with prophet

In [38]:
holiday_cols = [c for c in train_pred.columns if "Day" in c and "_" not in c]
prophet_feats_train = train_pred[["trend","holidays","weekly","yearly"] + holiday_cols]
prophet_feats_train.index = X_train1.index

In [39]:
X_train = train_sku[pred_cols].astype(np.float64)

In [42]:
X_train.merge(prophet_feats_train,left_index=True,right_index=True,how="left"\
                                )

,if_promo,if_competitor,min_sales_price,price_delta,ad_time,RETAIL_PRICE,trend,holidays,weekly,yearly,...,Independence Day (Observed),Juneteenth National Independence Day,Juneteenth National Independence Day (Observed),Labor Day,Martin Luther King Jr. Day,Memorial Day,New Year's Day,New Year's Day (Observed),Veterans Day,Veterans Day (Observed)
SALES_DATE,,,,,,,,,,,,,,,,,,,,,
2018-05-27,0.0,0.0,99.99,-99.99,365.0,99.99,30.791424,0.000000,-3.812675,-0.206047,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2018-05-28,0.0,0.0,99.99,-99.99,366.0,99.99,30.848895,6.321197,-5.498044,-0.139633,...,0.0,0.0,0.0,0.0,0.0,6.321197,0.0,0.0,0.0,0.0
2018-05-29,0.0,0.0,99.99,-99.99,367.0,99.99,30.906367,0.000000,-4.731260,-0.074781,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2018-05-30,0.0,0.0,99.99,-99.99,368.0,99.99,30.963838,0.000000,-3.471682,-0.006780,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2018-05-31,0.0,0.0,99.99,-99.99,369.0,99.99,31.021310,0.000000,-1.467007,0.068657,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-20,1.0,0.0,79.99,-79.99,1880.0,104.99,61.987862,0.000000,-3.471682,-1.306463,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2022-07-21,1.0,0.0,79.99,-79.99,1881.0,104.99,61.970304,0.000000,-1.467007,-1.189945,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2022-07-22,1.0,0.0,79.99,-79.99,1882.0,104.99,61.952746,0.000000,6.245405,-1.071637,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [21]:
train_pred.columns

Index(['ds', 'trend', 'yhat_lower', 'yhat_upper', 'trend_lower', 'trend_upper',
       'Christmas Day', 'Christmas Day_lower', 'Christmas Day_upper',
       'Christmas Day (Observed)', 'Christmas Day (Observed)_lower',
       'Christmas Day (Observed)_upper', 'Columbus Day', 'Columbus Day_lower',
       'Columbus Day_upper', 'Independence Day', 'Independence Day_lower',
       'Independence Day_upper', 'Independence Day (Observed)',
       'Independence Day (Observed)_lower',
       'Independence Day (Observed)_upper',
       'Juneteenth National Independence Day',
       'Juneteenth National Independence Day_lower',
       'Juneteenth National Independence Day_upper',
       'Juneteenth National Independence Day (Observed)',
       'Juneteenth National Independence Day (Observed)_lower',
       'Juneteenth National Independence Day (Observed)_upper', 'Labor Day',
       'Labor Day_lower', 'Labor Day_upper', 'Martin Luther King Jr. Day',
       'Martin Luther King Jr. Day_lower', 'Mart

In [45]:
x = [1,2,3]
[y if y<2 else 7 for y in x]

[1, 7, 7]

In [46]:
1-(0.9)**4

0.3439

In [47]:
(980)/(980+1950)

0.33447098976109213

In [49]:
a = np.ndarray([2,18,27,16,5,15,18])
(a-np.mean(a)).sum()

0.0

In [ ]:
(b-np.mean(b))